In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from autogluon.tabular import TabularPredictor
from catboost import CatBoostClassifier
from fisting import fit_catboost, fit_et, fit_lgbm, fit_rf, objective
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm

random.seed(0)
np.random.seed(0)


df_train = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")

In [ ]:
df_train

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,26169,20230310-7492-177993190,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,26170,20230625-16563-206126520,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,26171,20220624-7492-137587082,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,26172,20220427-7491-125459150,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


In [ ]:
def prepare_df(df):
    df = df.copy()
    df = df.drop(["Unnamed: 0", "№ брони"], axis=1)
    df["Дата бронирования"] = pd.to_datetime(df["Дата бронирования"])
    df["Заезд"] = pd.to_datetime(df["Заезд"])
    df["Выезд"] = pd.to_datetime(df["Выезд"])
    return df

In [ ]:
def preprocess_df(df):
    df = df.copy()
    df = prepare_df(df)

    return df

In [ ]:
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

In [ ]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


In [ ]:
df_train["target"] = df_train["Дата отмены"].apply(lambda x: int(pd.notna(x)))
df_train = df_train.drop(["Дата отмены", "Статус брони"], axis=1)

In [ ]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница,target
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1,0
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4,0
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3,0
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,2,0
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Номер «Студия»,3,4,0
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Номер «Стандарт»,2,2,0
26172,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Номер «Стандарт»,2,3,1


In [ ]:
predictor = TabularPredictor(
    label="target", problem_type="binary", eval_metric="roc_auc"
).fit(df_train, presets="high_quality", time_limit=560)

No path specified. Models will be saved in: "AutogluonModels/ag-20240922_124519"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Wed Aug  7 16:19:28 UTC 2024
CPU Count:          16
Memory Avail:       5.73 GB / 15.49 GB (37.0%)
Disk Space Avail:   184.77 GB / 468.09 GB (39.5%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is ena

In [ ]:
predictions = predictor.predict_proba(df_test)

In [ ]:
predictions

,0,1
0,0.913458,0.086542
1,0.900119,0.099881
2,0.946630,0.053370
3,0.915985,0.084015
4,0.855043,0.144957
...,...,...
11213,0.900134,0.099866
11214,0.784633,0.215367
11215,0.946576,0.053424
11216,0.946897,0.053103


In [ ]:
predictions[1].to_csv("autogluon_on_processed_data_2.csv", index=False, header=False)

: 

In [ ]:
submissiom.to_csv("4x_blending.csv", index=False, header=False)

In [ ]:
X_train = df_train.drop("target", axis=1)
y_train = df_train["target"]

In [ ]:
# for forests
# encoder = OneHotEncoder(drop="first", sparse_output=False)
# cat_columns = X_train.select_dtypes(include=["object"]).columns
# X_train_forest = pd.concat(
#     [
#         X_train.drop(cat_columns, axis=1),
#         pd.DataFrame(
#             encoder.fit_transform(X_train[cat_columns]),
#             columns=encoder.get_feature_names_out(),
#         ),
#     ],
#     axis=1,
# )
# df_test_forest = pd.concat(
#     [
#         df_test.drop(cat_columns, axis=1),
#         pd.DataFrame(
#             encoder.transform(df_test[cat_columns]),
#             columns=encoder.get_feature_names_out(),
#         ),
#     ],
#     axis=1,
# )

# # for lgbm
# X_train_lgbm = X_train.copy()
# df_test_lgbm = df_test.copy()
# for cat in cat_columns:
#     X_train_lgbm[cat] = X_train_lgbm[cat].astype("category")
#     df_test_lgbm[cat] = df_test_lgbm[cat].astype("category")

In [ ]:
SEARCH_BEST_PARAMS = True
N_TRIALS = 30
CAT_FEATURES = ["Способ оплаты", "Источник", "Категория номера"]
RANDOM_SEED = 42
EVAL_METRIC = "AUC"
EARLY_STOPPING = 50

In [ ]:
# from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm

# # Stratified train-test split
# X_train_select, X_test_select, y_train_select, y_test_select = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# # Define and fit CatBoost model
# model = CatBoostClassifier(
#         cat_features=CAT_FEATURES,
#         verbose=0,
#         eval_metric=EVAL_METRIC,
#         early_stopping_rounds=EARLY_STOPPING,
#     )

# # Perform feature selection
# selected_features = model.select_features(
#     X_train_select, y_train_select,
#     eval_set=(X_test_select,y_test_select),
#     features_for_select=X_train.columns.to_list(),  # selects from all features (indexing starts from 0)
#     num_features_to_select=15,   # number of features you want to keep
#     steps=30,                    # steps to remove features
#     verbose=True,
# )

In [ ]:
# X_train = X_train.drop(selected_features["eliminated_features_names"], axis=1)
# df_test = df_test.drop(selected_features["eliminated_features_names"], axis=1)

In [ ]:
# catboost is good on defaults
models_list = []
scores_list = []
y_pred = np.zeros(df_test.shape[0])
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
for i, (train_index, test_index) in enumerate(splitter.split(X_train, y_train)):
    X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
    X_fold_test, y_fold_test = X_train.iloc[test_index], y_train.iloc[test_index]

    model = CatBoostClassifier(
        cat_features=CAT_FEATURES,
        verbose=0,
        eval_metric=EVAL_METRIC,
        early_stopping_rounds=EARLY_STOPPING,
    )

    model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_test, y_fold_test))

    preds = model.predict_proba(X_fold_test)[:, 1]
    score = roc_auc_score(y_fold_test, preds)

    models_list.append(model)
    scores_list.append(score)

np.mean(scores_list), np.std(scores_list)

(0.8627034580046269, 0.014790056605410043)

In [ ]:
y_pred = np.zeros(df_test.shape[0])
for model in models_list:
    y_pred += model.predict_proba(df_test)[:, 1]

y_pred = y_pred / len(models_list)

submissiom = pd.DataFrame(y_pred)

In [ ]:
submissiom[0] = submissiom[0].apply(lambda x: int(x > 0.5))

In [ ]:
submissiom.value_counts()

0
0    10029
1     1189
Name: count, dtype: int64

In [ ]:
submissiom.to_csv("catboost_only_binary.csv", index=False, header=False)

In [ ]:
if SEARCH_BEST_PARAMS:
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
    )
    study.optimize(
        lambda trial: objective(trial, X_train_lgbm, y_train, "LightGBM"),
        n_trials=N_TRIALS - 15,
        show_progress_bar=True,
    )

[I 2024-09-21 21:56:14,953] A new study created in memory with name: no-name-0b654a55-3330-455c-949a-2538d7371036


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2024-09-21 21:58:19,366] Trial 0 finished with value: 0.8583994683558867 and parameters: {'boosting_type': 'dart', 'num_leaves': 64, 'max_depth': 4, 'learning_rate': 0.055238410897498764, 'feature_fraction': 0.4348501673009197, 'lambda_l1': 8.661895281603577, 'lambda_l2': 6.011549002420345, 'min_child_samples': 72, 'bagging_fraction': 0.41235069657748147, 'bagging_freq': 7}. Best is trial 0 with value: 0.8583994683558867.
[I 2024-09-21 21:58:25,624] Trial 1 finished with value: 0.8600574020899071 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 26, 'max_depth': 5, 'learning_rate': 0.16217936517334897, 'feature_fraction': 0.6591670111852694, 'lambda_l1': 2.913000172840221, 'lambda_l2': 6.118917094329073, 'min_child_samples': 18, 'bagging_fraction': 0.5752867891211308, 'bagging_freq': 3}. Best is trial 1 with value: 0.8600574020899071.
[I 2024-09-21 21:59:55,914] Trial 2 finished with value: 0.8602790611151823 and parameters: {'boosting_type': 'dart', 'num_leaves': 56, 'max_dep

KeyboardInterrupt: 

In [ ]:
if SEARCH_BEST_PARAMS:
    lgbm_best_params = study.best_params
else:
    lgbm_best_params = {
        "boosting_type": "dart",
        "num_leaves": 80,
        "max_depth": 8,
        "learning_rate": 0.02353980066877352,
        "feature_fraction": 0.7146404372369975,
        "lambda_l1": 2.9294801014827625,
        "lambda_l2": 4.301118582479693,
        "min_child_samples": 85,
        "bagging_fraction": 0.8207939948373919,
        "bagging_freq": 3,
    }

lgbm_best_value, lgbm_best_models = objective(
    optuna.trial.FixedTrial(lgbm_best_params),
    X_train_lgbm,
    y_train,
    "LightGBM",
    return_models=True,
)

print(f"Best LightGBM RMSE: {lgbm_best_value}\n\n")
print("Best LightGBM params:")
print(*[f"'{key}': {value}," for key, value in lgbm_best_params.items()], sep="\n")

Best LightGBM RMSE: 0.8611098264769588


Best LightGBM params:
'boosting_type': dart,
'num_leaves': 80,
'max_depth': 8,
'learning_rate': 0.02353980066877352,
'feature_fraction': 0.7146404372369975,
'lambda_l1': 2.9294801014827625,
'lambda_l2': 4.301118582479693,
'min_child_samples': 85,
'bagging_fraction': 0.8207939948373919,
'bagging_freq': 3,


In [ ]:
if SEARCH_BEST_PARAMS:
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
    )
    study.optimize(
        lambda trial: objective(trial, X_train_forest, y_train, "RandomForest"),
        n_trials=N_TRIALS,
        n_jobs=-1,
        show_progress_bar=True,
    )

[I 2024-09-21 22:34:28,072] A new study created in memory with name: no-name-b3153734-6743-45ee-b44b-751212972ae8


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2024-09-21 22:36:22,785] Trial 8 finished with value: 0.8543033831840481 and parameters: {'n_estimators': 94, 'max_depth': 15, 'min_samples_split': 48, 'min_samples_leaf': 45, 'max_features': 0.08342036906580597}. Best is trial 8 with value: 0.8543033831840481.
[I 2024-09-21 22:36:24,813] Trial 9 finished with value: 0.8594489110962236 and parameters: {'n_estimators': 86, 'max_depth': 8, 'min_samples_split': 42, 'min_samples_leaf': 36, 'max_features': 0.5906680603149131}. Best is trial 9 with value: 0.8594489110962236.
[I 2024-09-21 22:36:59,678] Trial 3 finished with value: 0.856373796720647 and parameters: {'n_estimators': 160, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 35, 'max_features': 0.23902912385045083}. Best is trial 9 with value: 0.8594489110962236.
[I 2024-09-21 22:37:34,469] Trial 7 finished with value: 0.8604503664788649 and parameters: {'n_estimators': 185, 'max_depth': 17, 'min_samples_split': 11, 'min_samples_leaf': 44, 'max_features': 0.16164208210

In [ ]:
# Вывод лучших параметров и метрик
if SEARCH_BEST_PARAMS:
    rf_best_params = study.best_params
else:
    rf_best_params = {
        "n_estimators": 425,
        "max_depth": 31,
        "min_samples_split": 33,
        "min_samples_leaf": 44,
        "max_features": 0.38331686104122775,
    }

rf_best_value, rf_best_models = objective(
    optuna.trial.FixedTrial(rf_best_params),
    X_train_forest,
    y_train,
    "RandomForest",
    return_models=True,
)

print(f"Best Random Forest RMSE: {rf_best_value}\n\n")
print("Best Random Forest params:")
print(*[f"'{key}': {value}," for key, value in rf_best_params.items()], sep="\n")

Best Random Forest RMSE: 0.8612870466853071


Best Random Forest params:
'n_estimators': 425,
'max_depth': 31,
'min_samples_split': 33,
'min_samples_leaf': 44,
'max_features': 0.38331686104122775,


In [ ]:
if SEARCH_BEST_PARAMS:
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
    )
    study.optimize(
        lambda trial: objective(trial, X_train_forest, y_train, "ExtraTrees"),
        n_trials=N_TRIALS - 15,
        n_jobs=1,
        show_progress_bar=True,
    )

[I 2024-09-21 22:44:54,418] A new study created in memory with name: no-name-a75b9a82-c594-402a-a038-c6cc79c6c16f


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2024-09-21 22:44:59,396] Trial 0 finished with value: 0.8551317486861899 and parameters: {'n_estimators': 218, 'max_depth': 31, 'min_samples_split': 37, 'min_samples_leaf': 30, 'max_features': 0.15601864044243652}. Best is trial 0 with value: 0.8551317486861899.
[I 2024-09-21 22:45:01,395] Trial 1 finished with value: 0.846770334777635 and parameters: {'n_estimators': 120, 'max_depth': 3, 'min_samples_split': 44, 'min_samples_leaf': 31, 'max_features': 0.7080725777960455}. Best is trial 0 with value: 0.8551317486861899.
[I 2024-09-21 22:45:03,062] Trial 2 finished with value: 0.8585579156835859 and parameters: {'n_estimators': 59, 'max_depth': 32, 'min_samples_split': 42, 'min_samples_leaf': 11, 'max_features': 0.18182496720710062}. Best is trial 2 with value: 0.8585579156835859.
[I 2024-09-21 22:45:06,712] Trial 3 finished with value: 0.8599164836062212 and parameters: {'n_estimators': 132, 'max_depth': 11, 'min_samples_split': 27, 'min_samples_leaf': 22, 'max_features': 0.29122914

In [ ]:
# Вывод лучших параметров и метрик
if SEARCH_BEST_PARAMS:
    et_best_params = study.best_params
else:
    et_best_params = {
        "n_estimators": 497,
        "max_depth": 13,
        "min_samples_split": 33,
        "min_samples_leaf": 17,
        "max_features": 0.5950385565501397,
    }

et_best_value, et_best_models = objective(
    optuna.trial.FixedTrial(et_best_params),
    X_train_forest,
    y_train,
    "ExtraTrees",
    return_models=True,
)

print(f"Best Extra Trees RMSE: {et_best_value}\n\n")
print("Best Extra Trees params:")
print(*[f"'{key}': {value}," for key, value in et_best_params.items()], sep="\n")

Best Extra Trees RMSE: 0.8608913874445427


Best Extra Trees params:
'n_estimators': 497,
'max_depth': 13,
'min_samples_split': 33,
'min_samples_leaf': 17,
'max_features': 0.5950385565501397,


In [ ]:
catboost_y_pred = np.zeros(df_test.shape[0])

for model in models_list:
    catboost_y_pred += model.predict_proba(df_test)[:, 1]
catboost_y_pred = catboost_y_pred / len(models_list)

lgbm_y_pred = np.zeros(df_test_lgbm.shape[0])

for model in lgbm_best_models:
    lgbm_y_pred += model.predict_proba(df_test_lgbm)[:, 1]
lgbm_y_pred = lgbm_y_pred / len(models_list)


rf_y_pred = np.zeros(df_test_forest.shape[0])

for model in rf_best_models:
    rf_y_pred += model.predict_proba(df_test_forest)[:, 1]
rf_y_pred = rf_y_pred / len(models_list)


et_y_pred = np.zeros(df_test_forest.shape[0])

for model in et_best_models:
    et_y_pred += model.predict_proba(df_test_forest)[:, 1]
et_y_pred = et_y_pred / len(models_list)

In [ ]:
final = np.zeros(df_test.shape[0])

final += catboost_y_pred + lgbm_y_pred + rf_y_pred + et_y_pred
final = final / 4

In [ ]:
submissiom = pd.DataFrame(final)
submissiom.to_csv("4x_blending.csv", index=False, header=False)